In [4]:
import os
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#import pyarrow as pa
#import datetime

# Define paths
wea_files = ['LC_2022Q1.csv', 'LC_2022Q2.csv', 'LC_2022Q3.csv', 'LC_2022Q4.csv']
id_list = ['439', '440', '441', '910', '442', '443', '444', '445', '324'] # Same order as 1-9 in project.pdf

# Load with polars and remove noise stations 7 8 9
noise_loc = pl.read_csv('../noise_meta.csv')\
            .slice(0,6)
wea_loc = pl.read_csv('../01_Metadata_v2.csv')
wea_prep = pd.read_csv('out/wea_prep.csv')

In [36]:
def loc_filter(noise, wea):
    # Find the range of longitude and latitude values in noise
    lon_min, lon_max = noise['LON'].min(), noise['LON'].max()
    lat_min, lat_max = noise['LAT'].min(), noise['LAT'].max()

    # Filter wea
    wea_filtered = wea.filter((wea['LON'] > lon_min) & (wea['LON'] < lon_max) & (wea['LAT'] > lat_min) & (wea['LAT'] < lat_max))
    
    return wea_filtered

def wea_preprocess(folder_path, file_names, filter):
    # Initialize dataframe with 1 line makes polars works
    #wea_prep = pl.DataFrame()
    wea_prep = pl.DataFrame({
                'ID':['LC-null'],
                'month':[0],
                'day':[0],
                'hour':[0],
                'rain':[0],
                'temp':[0]
                }, schema=[    
                    ("ID", pl.Utf8),
                    ("month", pl.Int32),
                    ("day", pl.Int32),
                    ("hour", pl.Int32),
                    ("rain", pl.Float64),
                    ("temp", pl.Float64)])
    # Cast rain to utf8, filter NA, cast back to f64 (no idea what NA as i64 is for polars)
    for file_name in file_names:
        wea = pl.read_csv(folder_path + file_name, 
                    columns=[1,12,13,14,6,20], 
                    new_columns=['ID','month','day','hour','rain','temp'],
                    infer_schema_length=1500000)\
                .filter(pl.col('ID').is_in(filter['ID'].unique().to_list()))\
                .filter(pl.col('rain').cast(pl.Utf8) != 'NA')\
                .filter(pl.col('temp') != 'NA')\
                .with_columns(
                    pl.col('month').cast(pl.Int32,strict=False),
                    pl.col('day').cast(pl.Int32,strict=False),
                    pl.col('hour').cast(pl.Int32,strict=False),
                    pl.col('rain').cast(pl.Float64,strict=False),
                    pl.col('temp').cast(pl.Float64, strict=False))
    wea_prep.extend(wea)
    wea_prep = wea_prep.slice(1, None).to_pandas()       
    
    # create a new column 'weekend' based on the 'month' and 'day' columns
    wea_prep['weekend'] = pd.to_datetime('2022-' + wea_prep['month'].astype(str) + '-' + wea_prep['day'].astype(str))\
                            .dt.day_name()\
                            .apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
    wea_prep = pl.from_pandas(wea_prep).with_columns(pl.col('weekend').cast(pl.Int32,strict=False))
    # Aggregate on: mean temp, max rain, per hour
    wea_prep = wea_prep.groupby(['month', 'day', 'hour'])\
                .agg(
                    temp = pl.mean('temp'),
                    rain = pl.max('rain')
                )
    return wea_prep


In [ ]:
wea1 = pl.read_csv('../weather/' + 'LC_2022Q1.csv', 
                    columns=[1,12,13,14,6,20], 
                    new_columns=['ID','month','day','hour','rain','temp'],
                    infer_schema_length=1500000)
print(wea1)

In [37]:
wea_prep = wea_preprocess('../weather/', wea_files, loc_filter(noise_loc, wea_loc))
#wea_prep.to_csv('out/wea_prep.csv', index=False)
print(wea_prep)

C:\Users\camd1\AppData\Local\Temp\ipykernel_15776\2715302889.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wea_prep['weekend'] = pd.to_datetime('2022-' + wea_prep['month'].astype(str) + '-' + wea_prep['day'].astype(str))\


DateParseError: Unknown datetime string format, unable to parse: 2022-nan-10, at position 0

In [27]:
noise_event = pl.read_csv(os.path.join(basepath, '2. noise events/', 'csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv'), separator=';')

In [ ]:
print(noise_event.sample(20))

In [33]:
noise_event = noise_event.select([
        pl.col('result_timestamp').str.extract(r'(\d{2})/(\d{2})/\d{4}\s(\d{2}):', 1).cast(pl.Int32).alias('month'),
        pl.col('result_timestamp').str.extract(r'(\d{2})/(\d{2})/\d{4}\s(\d{2}):', 2).cast(pl.Int32).alias('day'),
        pl.col('result_timestamp').str.extract(r'(\d{2})/(\d{2})/\d{4}\s(\d{2}):', 3).cast(pl.Int32).alias('hour'),
        pl.col('noise_event_laeq_primary_detected_class').alias('event_type').fill_null("Nothing")
        ])

In [ ]:
print(noise_event.sample(20))